# A South American Grocery Store needs prediction of how many items they would sell on certain dates. (NW RMSLE)

In [11]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import os
import math
import re
import pandas as pd
import numpy as np
from sklearn import metrics
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [3]:
PATH = "data/groceries/"

In [4]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False,
                    parse_dates = ['date'])

In [5]:
df_raw.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN


## Here we have 6 columns but for hundreds of thousands of items(too big for most csv readers).

## The main purpose of this project is to learn how to separate the data into multiple trees and then test for at least 50% accuracy

In [6]:
df_raw['date'].head(3)

0   2013-01-01
1   2013-01-01
2   2013-01-01
Name: date, dtype: datetime64[ns]

# Feature Engineering
## Starting with df_raw['date']. 

In [7]:
def add_datepart(df, fldname):
    fld= df[fldname] 
    #not df.fldname, that would just grab a field that states 'fldname'. also safer and should do more than this.
    targ_pre = re.sub('[Dd]ate$', '', fldname)#regex removes 'date' from fldname
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek','Dayofyear',#goes through each string
              'Is_month_end','Is_month_start', 'Is_quarter_end', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
        #tries to find an attribute with the same name inside each object(like Year attr)
    df[targ_pre+'Elapsed'] = (fld - fld.min()).dt.days
    df.drop(fldname, axis=1, inplace=True)

In [9]:
fld = df_raw.date

In [12]:
add_datepart(df_raw, 'date')

In [56]:
def isHoliday(day, month):#later add year to this.
    fixedHolDates = ([1,1], [7,9],[25,12],[28,7],[20,7],[12,11],[8,12])#in [day, month]
    df_date = [day,month]
    if all(elem in fixedHolDates for elem in df_date):
        return True;
    return False;

In [57]:
a = isHoliday(df_raw['Day'][3],df_raw['Month'][3])
a

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()